<a href="https://colab.research.google.com/github/Clei21/Project_REMARS/blob/main/Project__Remars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# NASA Space Apps Challenge 2025
## **Team:** Tambackins  
## **Project:** REMARS — *Recycling & Manufacturing for Mars Sustainability*  

# **Team Members:**  
# - Leonor Joana Ramos Oliveira  
# - Cleidiana Manoel Alves  
# - Rayane Farias
# - Jonatas da Silva Santos

#  **Mission Context:**  
# During long-term missions to Mars, crews generate large amounts of inorganic waste such as packaging, textiles, and structural materials. REMARS proposes an intelligent recycling system that classifies, transforms, and reuses Martian mission waste to produce 3D printing filaments and modular parts, reducing resupply needs and increasing mission autonomy.
#

#  **Main Components:**
# 1. Image-based waste material classification  
# 2. Recycling pipeline planner (energy/time estimates)  
# 3. Generation of printable OpenSCAD parts (blocks and origami panels)  
# 4. Production queue simulator  
# 5. Interactive Gradio interface for demo




#Imports

In [ ]:
!pip -q install trimesh pyglet


In [ ]:
import os
import uuid
import json
import time
import tempfile
from pathlib import Path
import cv2
import gradio as gr
import numpy as np
from PIL import Image
import matplotlib
matplotlib.use("Agg")
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Labels and Energy Reference

In [ ]:
LABELS = ["plastic", "metal", "textile", "composite"]

ENERGY_REF = {
    "SHRED_Wh_per_kg": 120,
    "EXTRU_Wh_per_kg": 600,
    "PRINT_Wh_per_kg": 450,
    "METAL_kWh_per_kg": 5.0,
    "TEXTILE_Wh_per_kg": 180
}

last_material = None

#Heuristic Material Classifier

In [ ]:
def classify_material(img: Image.Image):
    global last_material
    im = np.array(img.convert("RGB"))
    hsv = cv2.cvtColor(im, cv2.COLOR_RGB2HSV)
    sat = float(hsv[..., 1].mean())
    val = float(hsv[..., 2].mean())
    edges = float(cv2.Canny(cv2.cvtColor(im, cv2.COLOR_RGB2GRAY), 80, 200).mean())
    score = {
        "plastic": 0.6 * sat + 0.2 * val - 0.1 * edges,
        "metal":   0.7 * edges + 0.2 * val - 0.2 * sat,
        "textile": 0.6 * sat + 0.1 * edges - 0.1 * val,
        "composite": 0.3 * sat + 0.3 * edges + 0.1 * val
    }
    label = max(score, key=score.get)
    last_material = label
    score = {k: float(v) for k, v in score.items()}
    return label, json.dumps(score, indent=2)

#Recycling Pipeline Planner

In [ ]:

def plan_pipeline(material: str, mass_kg: float, isru_pct: float):
    mass_kg = max(0.01, float(mass_kg))
    isru_pct = max(0.0, min(80.0, float(isru_pct)))
    plan = []
    energy_Wh = 0.0
    if material == "plastic":
        plan = ["SEP", "SHRED", "EXTRU", "PRINT3D"]
        energy_Wh += ENERGY_REF["SHRED_Wh_per_kg"] * mass_kg
        energy_Wh += ENERGY_REF["EXTRU_Wh_per_kg"] * mass_kg
        energy_Wh += ENERGY_REF["PRINT_Wh_per_kg"] * mass_kg
    elif material == "metal":
        plan = ["SEP", "METAL"]
        energy_Wh += ENERGY_REF["METAL_kWh_per_kg"] * 1000 * mass_kg
    elif material == "textile":
        plan = ["SEP", "TEXTILE"]
        energy_Wh += ENERGY_REF["TEXTILE_Wh_per_kg"] * mass_kg
    else:
        plan = ["SEP", "SHRED", "EXTRU", "PRINT3D"]
        energy_Wh += (ENERGY_REF["SHRED_Wh_per_kg"] +
                      ENERGY_REF["EXTRU_Wh_per_kg"] +
                      ENERGY_REF["PRINT_Wh_per_kg"]) * mass_kg
    comp_ratio = isru_pct / 100.0
    effective_mass = mass_kg * (1.0 + comp_ratio)
    throughput_kg_h = {"SHRED": 6.0, "EXTRU": 1.2, "PRINT3D": 0.3, "METAL": 0.8, "TEXTILE": 2.0}
    def step_time(step: str) -> float:
        if step in throughput_kg_h:
            return effective_mass / throughput_kg_h[step]
        return 0.05
    total_h = sum(step_time(s) for s in plan)
    return {
        "pipeline": plan,
        "mass_total_kg": round(effective_mass, 3),
        "energy_Wh": int(energy_Wh),
        "est_time_h": round(total_h, 2),
        "isru_pct": isru_pct,
        "material": material
    }

#OpenSCAD Generator: LEGO Block and Origami Panel

In [ ]:
def _write_scad(basename: str, content: str) -> str:
    d = tempfile.mkdtemp()
    p = os.path.join(d, basename)
    with open(p, "w", encoding="utf-8") as f:
        f.write(content)
    return p

def _save_fig(fig) -> str:
    d = tempfile.mkdtemp()
    p = os.path.join(d, f"preview_{uuid.uuid4().hex}.png")
    fig.savefig(p, dpi=150, bbox_inches="tight")
    plt.close(fig)
    return p

def scad_block_lego(w_cm: float = 20, h_cm: float = 20, t_cm: float = 3, pin: bool = True) -> str:
    w = int(w_cm * 10)
    h = int(h_cm * 10)
    t = int(t_cm * 10)
    pins = ""
    if pin:
        step = 40
        for x in range(step // 2, int(w), step):
            for y in range(step // 2, int(h), step):
                pins += f"translate([{x},{y},{t}]) cylinder(h=8, r=7, $fn=48);\n"
    scad = f"""
module block() {{
  cube([{w},{h},{t}], center=false);
  {pins}
}}
block();
"""
    name = f"block_{uuid.uuid4().hex}.scad"
    return _write_scad(name, scad)

def scad_panel_origami(w_cm: float = 20, h_cm: float = 20, t_mm: float = 2, grid: int = 6) -> str:
    w = int(w_cm * 10)
    h = int(h_cm * 10)
    t = int(t_mm)
    lines = ""
    dx = w / grid
    dy = h / grid
    for i in range(grid + 1):
        lines += f"translate([0,{i*dy},0]) cube([{w},1,{t}], false);\n"
        lines += f"translate([{i*dx},0,0]) cube([1,{h},{t}], false);\n"
    scad = f"""
module panel() {{
  difference() {{
    cube([{w},{h},{t}], center=false);
    translate([0,0,0]) union() {{
      {lines}
    }}
  }}
}}
panel();
"""
    name = f"panel_origami_{uuid.uuid4().hex}.scad"
    return _write_scad(name, scad)

def preview_block_png(w_cm: float = 20, h_cm: float = 20, t_cm: float = 3, pin: bool = True) -> str:
    w = float(w_cm * 10)
    h = float(h_cm * 10)
    t = float(t_cm * 10)
    fig = plt.figure(figsize=(6, 4))
    ax = fig.add_subplot(111, projection="3d")
    ax.bar3d(0, 0, 0, w, h, t, shade=True)
    if pin:
        step = 40
        theta = np.linspace(0, 2 * np.pi, 48)
        z = np.linspace(t, t + 8, 2)
        tt, zz = np.meshgrid(theta, z)
        r = 7.0
        for x in range(step // 2, int(w), step):
            for y in range(step // 2, int(h), step):
                xx = r * np.cos(tt) + float(x)
                yy = r * np.sin(tt) + float(y)
                ax.plot_surface(xx, yy, zz, linewidth=0, antialiased=False)
    ax.set_box_aspect((w, h, t + 8))
    ax.set_axis_off()
    return _save_fig(fig)

def preview_panel_png(w_cm: float = 20, h_cm: float = 20, t_mm: float = 2) -> str:
    w = float(w_cm * 10)
    h = float(h_cm * 10)
    t = float(t_mm)
    fig = plt.figure(figsize=(6, 4))
    ax = fig.add_subplot(111, projection="3d")
    ax.bar3d(0, 0, 0, w, h, t, shade=True)
    ax.set_box_aspect((w, h, t))
    ax.set_axis_off()
    return _save_fig(fig)

def gen_block_assets():
    return scad_block_lego(), preview_block_png()

def gen_panel_assets():
    return scad_panel_origami(), preview_panel_png()


#Queue Simulator

In [ ]:
JOBS = []

def enqueue_job(material: str, mass_kg: float, isru_pct: float, piece: str):
    plan = plan_pipeline(material, mass_kg, isru_pct)
    job = {
        "id": int(time.time() * 1000),
        "material": material,
        "mass_kg": float(mass_kg),
        "isru_pct": float(isru_pct),
        "piece": piece,
        "plan": plan,
        "status": "queued"
    }
    JOBS.append(job)
    return json.dumps(job, indent=2)

def list_jobs():
    return json.dumps(JOBS[-10:], indent=2)


#Gradio Interface

In [ ]:
def get_last_material():
    return (last_material or "plastic")

with gr.Blocks(theme="base") as demo:
    gr.Markdown("# Mars FabLab — Recycling & Origami")
    gr.Markdown("Classify waste, plan recycling pipeline, generate SCAD parts and simulate queue.")

    with gr.Tab("Classify Material"):
        img = gr.Image(type="pil", label="Upload waste image")
        out_label = gr.Textbox(label="Predicted material")
        out_score = gr.Code(label="Scores")
        gr.Button("Classify").click(classify_material, img, [out_label, out_score])

    with gr.Tab("Plan Recycling"):
        mat = gr.Dropdown(LABELS, label="Material", value="plastic")
        mass = gr.Number(label="Mass (kg)", value=1.0, precision=2)
        isru = gr.Slider(0, 80, value=20, step=5, label="ISRU blend (%)")
        gr.Button("Use last detected material").click(get_last_material, outputs=mat)
        plan_out = gr.JSON(label="Plan")
        gr.Button("Generate plan").click(plan_pipeline, [mat, mass, isru], plan_out)

    with gr.Tab("Generate Part (SCAD)"):
        with gr.Row():
            btn1 = gr.Button("LEGO Block 20×20 cm")
            scad1 = gr.File(label="block.scad")
            prev1 = gr.Image(label="Preview", type="filepath")
        with gr.Row():
            btn2 = gr.Button("Origami Panel 20×20 cm")
            scad2 = gr.File(label="panel_origami.scad")
            prev2 = gr.Image(label="Preview", type="filepath")
        btn1.click(gen_block_assets, outputs=[scad1, prev1])
        btn2.click(gen_panel_assets, outputs=[scad2, prev2])

    with gr.Tab("Simulate Production"):
        mat2 = gr.Dropdown(LABELS, label="Material", value="plastic")
        mass2 = gr.Number(label="Mass (kg)", value=0.5, precision=2)
        isru2 = gr.Slider(0, 80, value=10, step=5, label="ISRU blend (%)")
        piece = gr.Dropdown(["block_20cm", "panel_origami"], value="block_20cm", label="Part")
        out_job = gr.Code(label="Queued job")
        out_queue = gr.Code(label="Queue (last 10)")
        gr.Button("Add to queue").click(enqueue_job, [mat2, mass2, isru2, piece], out_job)
        gr.Button("Show queue").click(list_jobs, None, out_queue)

#Launch

In [ ]:
demo.launch(debug=False)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b879567508f560d9ab.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
